# Sources Quality Rank

## Intro

The objective of this project is to build a recommendation system to detect the best traffic sources (apps or websites) where a specific app in question can be promoted. An online advertising company X has a lot of historical information for a big number of apps that they promoted, including:

- App vertical of the promoted app (Games, Shopping, Transportation, etc.)
- Country where the app was promoted
- Operating System of the app
- All traffic sources that were used to promote the app
- What were the results of promotion in terms of volumes (number of conversions) and traffic quality

As a proxy for traffic quality the following metrics are used:

- KPI reach (how good was the traffic source in terms of reaching client's objective for the campaign)
- Fraud Index (how likely the conversions brought by the traffic source are fraudulent, based on a third-party fraud detection tool)
- Early Conversions (how many installs are coming within the first 2 hours after the click which is considered to be normal user behaviour). This will be used to calculate ClickSpam Index, which is another fraud indicator not accounted for by Fraud Index.

**KPI Reach** 

Depending on the Client, each app will have a different KPI goal, which can be formulated in one of the following ways:

- **Percentage**: out of all acquired users, how many do you expect to convert to a specific in-app event (for example, first ride)

*For example, if your KPI goal for converting from install to first ride is 20%, you expect at least 20 out of 100 users who installed the app to book a first ride with your taxi app. If a specific traffic source generated 100 installs, and only 10 users booked a first ride, the Percentage of this source is 10%, and therefore they only reached 50% of your goal (KPI Reach = 50%).*

- **CPE (Cost per Event)**: what is the maximum amount you are willing to pay for a user who converted to a specific in-app event (for example, first ride)

*Let's say you are running a CPI user acquisition campaign where you pay 1 USD for each acquired user. But you are mostly interested in users who book a first ride in your app, and you are willing to pay no more than 4 USD per paying user (i.e. user who booked a first ride). So if a traffic source generates 100 installs (100 USD cost for you) and out of those 25 users book a first ride, your CPE (cost per first ride event) would be 100 USD / 25 = 4 USD exactly equal to your KPI (KPI Reach = 100%).
If more users booked a first ride, the CPE would be lower, and therefore the KPI goal would be overreached. On the contrary, if less users booked a first ride, each paying user would have costed you more than 4 USD, and therefore your KPI goal would not be reached.*

- **ROI (Return on Investment)**: what return on investment you expect for a specific campaign

*ROI = Revenue / Cost, where revenue is all the amount spent by the acquired users in your app, and Cost is how much you spent on user acquisition. So if you acquired 100 users paying 1 USD for each install (your cost is 100 USD), and those users made several purchases that resulted in a total spend of 120 USD, your ROI is 120%.*

**NOTE**: all the names, such as sourceName, country, OS, app vertical were randomly generated and assigned.

## Project Structure

The project is divided into three parts:

1. Extract and Transform the data
2. Upload the resulting table to Google Sheets
3. Build Tableau dashboard based on the data

![alt text](pqr_dataflow.png "Title")

In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import pygsheets

## Part 1. Extract & Transform

In this part we want to get all the necessary data in order to be able to build the Quality Rank.

A. The data should be grouped by the following dimensions:

- SourceId : the id of the app / website that is used as a traffic source
- PublisherId : the subid of the app / website that is used as a traffic source, this can be thought of placement
- Country : where the app is being promoted
- OS : operating system of the app
- lineCategory : App Vertical of the app that was promoted

B. The following metrics will be generated: 

- Conversions 
- earlyConversions: how many conversions happened within the first 2 hours after the click
- kpiGoalReach: what % of client's goal was reached
- has_goal: whether a campaign had a KPI goal configured
- fraudIndex : fraud index assigned by a third-party fraud detection tool

### SQL query to combine the necessary data

We have 4 input tables: 

- Conv : this table contains the amount of conversions and earlyConversions for each combination of the specified dimensions (+ lineId dimension which indicates the id of promotional campaign ran by the advertising company X)
- Kpis : this table contains the KPI goal configured for each promotional campaign (lineId)
- Events : this table contains tha amount of all post-install events that happened on a specific campaign after the user installed the app
- Fraud : this table contains Fraud Index assigned to each combination of the specified dimensions by the third-party fraud detection tool

A number of intermediary tables will be generated in order to combine the data in the desired way.

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [3]:
# connect to SQLite database with the data
database = "C:\\sqlite\db\pqr.db"
conn = create_connection(database)

In [4]:
# query that combines all the information from multiple tables
sql = """WITH
    c AS (
        SELECT
            publisherId,
            lineId,
            sourceId,
            countryName,
            os,
            lineCategory,
            SUM (conversions) AS conversions,
            SUM (earn) AS earn,
            SUM (earlyConversions) AS earlyConversions
        FROM
            conv
        WHERE
            countryName IS NOT NULL AND
            os IS NOT NULL AND
            lineCategory IS NOT NULL
        GROUP BY
            publisherId,
            lineId,
            sourceId,
            countryName,
            os,
            lineCategory
    ),
    g AS (
        SELECT
            id AS lineId,
            event AS eventName,
            kpi_type AS goalType,
            kpi_goal AS goalValue
        FROM
            kpis
    ),
    e AS (
        SELECT
            publisherId,
            lineId,
            sourceId,
            event,
            countryName,
            os,
            lineCategory,
            SUM (eventCount) AS eventCount,
            SUM (clientRevenue) AS clientRevenue
        FROM
            events
        WHERE
            countryName IS NOT NULL AND
            os IS NOT NULL AND
            lineCategory IS NOT NULL
        GROUP BY
            publisherId,
            lineId,
            sourceId,
            countryName,
            os,
            lineCategory,
            event
    ),
    k AS (
        SELECT
            c.lineId,
            c.sourceId,
            c.publisherId,
            c.countryName,
            c.os,
            c.lineCategory,
            c.conversions,
            c.earlyConversions,
            e.eventCount,
            g.eventName,
            g.goalType,
            g.goalValue,
            CAST (c.earn AS DOUBLE) / e.eventCount AS CPE,
            CAST (e.clientRevenue AS DOUBLE) / c.earn * 100 AS ROI,
            e.eventCount / CAST (c.conversions AS DOUBLE) * 100 AS Percentage
        FROM
            c
        LEFT JOIN
            g
        ON
            c.lineId = g.lineId
        LEFT JOIN
            e
        ON
            c.lineId = e.lineId AND
            c.sourceId = e.sourceId AND
            c.publisherId = e.publisherId AND
            c.countryName = e.countryName AND
            c.os = e.os AND
            c.lineCategory = e.lineCategory AND
            g.eventName = e.event
    ), 
    kpl AS (
            SELECT 
                lineid,
                sourceId,
                publisherId,
                countryName,
                os,
                lineCategory,
                SUM (conversions) AS conversions,
                SUM (earlyConversions) AS earlyConversions,
                CASE WHEN goalType = 'Percentage'
                THEN (Percentage / goalValue) * 100
                ELSE CASE WHEN goalType = 'ROI' 
                     THEN (ROI / goalValue) * 100
                     ELSE (goalValue / CPE) * 100 
                     END
                END
                AS kpi_reach_abs
            FROM
                k 
            WHERE 
                eventName IS NOT NULL
            GROUP BY 
            lineid,
            sourceId,
            publisherId,
            countryName,
            os,
            lineCategory
    ),
    p AS (
        SELECT
            sourceId,
            publisherId,
            countryName,
            os,
            lineCategory,
            SUM (conversions) AS conversions, 
            SUM (earlyConversions) AS earlyConversions,
            SUM (IFNULL(kpi_reach_abs,0) * conversions) / SUM (conversions) AS kpiGoalReach
        FROM kpl
        GROUP BY
            sourceId,
            publisherId,
            countryName,
            os,
            lineCategory
    ),
    o AS (    
        SELECT
            sourceId,
            publisherId,
            countryName,
            os,
            lineCategory,
            SUM (conversions) AS conversions,
            SUM (earlyConversions) AS earlyConversions
        FROM
            k
        WHERE
            eventName IS NULL
        GROUP BY
            publisherId,
            sourceId,
            countryName,
            os,
            lineCategory
    ),
    po AS (
        SELECT
            COALESCE (p.sourceId, o.sourceId) AS sourceId,
            COALESCE (p.publisherId, o.publisherId) AS publisherId,
            COALESCE (p.countryName, o.countryName) AS countryName,
            COALESCE (p.os, o.os) AS os,
            COALESCE (p.lineCategory, o.lineCategory) AS lineCategory,
            COALESCE (p.conversions, 0) + COALESCE (o.conversions, 0) AS conversions,
            COALESCE (p.earlyConversions, 0) + COALESCE (o.earlyConversions, 0) AS earlyConversions,
            p.kpiGoalReach AS kpiGoalReach,
            1 AS has_goal
        FROM
            p
        LEFT JOIN
            o
        ON
            p.sourceId = o.sourceId AND
            p.publisherId = o.publisherId AND
            p.countryName = o.countryName AND
            p.os = o.os AND
            p.lineCategory = o.lineCategory
        UNION ALL
        SELECT
            COALESCE (p.sourceId, o.sourceId) AS sourceId,
            COALESCE (p.publisherId, o.publisherId) AS publisherId,
            COALESCE (p.countryName, o.countryName) AS countryName,
            COALESCE (p.os, o.os) AS os,
            COALESCE (p.lineCategory, o.lineCategory) AS lineCategory,
            COALESCE (p.conversions, 0) + COALESCE (o.conversions, 0) AS conversions,
            COALESCE (p.earlyConversions, 0) + COALESCE (o.earlyConversions, 0) AS earlyConversions,
            p.kpiGoalReach AS kpiGoalReach,
            0 AS has_goal
        FROM
            o
        LEFT JOIN
            p
        ON
            p.sourceId = o.sourceId AND
            p.publisherId = o.publisherId AND
            p.countryName = o.countryName AND
            p.os = o.os AND
            p.lineCategory = o.lineCategory
        WHERE p.publisherId IS NULL
    ), 
    f AS (
        SELECT
            sourceId,
            countryName,
            os,
            lineCategory
        FROM
            po
        GROUP BY
            sourceId,
            countryName,
            os,
            lineCategory
        HAVING
            SUM (conversions) >= 100
    ),
    fr AS (
        SELECT
            affiliate_id AS sourceId,
            aff_sub3 AS publisherId,
            country AS countryName,
            os,
            aff_sub2 AS lineCategory,
            score AS fraudIndex
        FROM
            fraud
    )
SELECT
    po.sourceId,
    po.publisherId,
    po.countryName,
    po.os,
    po.lineCategory,
    po.conversions,
    po.earlyConversions,
    po.kpiGoalReach,
    po.has_goal,
    fr.fraudIndex
FROM
    po
INNER JOIN
    f
ON
    po.sourceId = f.sourceId AND
    po.countryName = f.countryName AND
    po.os = f.os AND
    po.lineCategory = f.lineCategory
LEFT JOIN
    fr
ON
    po.sourceId = fr.sourceId AND
    po.publisherId = fr.publisherId AND
    po.countryName = fr.countryName AND
    po.os = fr.os AND
    po.lineCategory = fr.lineCategory"""

In [8]:
# load the query results into pandas dataframe
df = pd.read_sql(sql, conn)
df.sourceId = pd.to_numeric(df.sourceId)

df.head()

,sourceId,publisherId,countryName,os,lineCategory,conversions,earlyConversions,kpiGoalReach,has_goal,fraudIndex
0,1001,422950,France,Android,Transportation,222,222,0.0,1,8
1,1001,426502,France,Android,Transportation,78,78,0.0,1,17
2,1001,426502,France,iOS,Finance,206,206,0.0,1,47
3,1001,362381,France,iOS,Shopping,144,137,NaN,0,20
4,1001,403248,France,iOS,Travel&Local,37,37,0.0,1,1


In [9]:
# loading a list of randomly generated appNames
pubs = pd.read_csv('pubs.csv', names=['sourceId', 'appName'])

pubs.head()

,sourceId,appName
0,1001,com.futbin
1,1020,wp.wattpad
2,1030,tv.online53
3,1048,com.nzn.tdg
4,1051,co.giftloop


In [19]:
# adding appNames to the dataset
final = df.merge(pubs, on='sourceId')
final.head()

,sourceId,publisherId,countryName,os,lineCategory,conversions,earlyConversions,kpiGoalReach,has_goal,fraudIndex,appName
0,1001,422950,France,Android,Transportation,222,222,0.0,1,8,com.futbin
1,1001,426502,France,Android,Transportation,78,78,0.0,1,17,com.futbin
2,1001,426502,France,iOS,Finance,206,206,0.0,1,47,com.futbin
3,1001,362381,France,iOS,Shopping,144,137,NaN,0,20,com.futbin
4,1001,403248,France,iOS,Travel&Local,37,37,0.0,1,1,com.futbin


## Part 2. Load 
### Uploading the data to Google Sheets

To upload the resulting dataframe to Google Sheets, follow the instructions provided in this [blog post](https://erikrood.com/Posts/py_gsheets.html) using [pygsheets library](https://pygsheets.readthedocs.io/en/stable/).

In [21]:
# authorization
gdocs_conn = pygsheets.authorize(service_file='/Users/admif/Desktop/pqr/credentials.json')

In [22]:
# open Google Sheets
sheets = gdocs_conn.open('PQR')
# select the first sheet
work_sheet = sheets[0]
# write dataframe to google sheets
work_sheet.set_dataframe(final,(1,1), fit=True)

## Part 3. Building Tableau Dashboard

Tableau dashboard will use the dataframe stored in Google Sheets as a data source. 

Several calculated fields are added at this point in order to build the final Quality Score:

- Clickspam Index

`(SUM([Conversions])-SUM([Early Conversions]))*100/SUM([Conversions])`

Ranges from 0 to 100 and indicates the proportion of conversions that came after the first 2 hours after the click. The higher the Clickspam Index, the more likely that the source is doing clickspamming type of fraud.

- KPI Reach

`IF (SUM ([Kpi Goal Reach] * [Conversions]) / 
SUM ([Conversions] * [Has Goal])) > 100
THEN 100
ELSE SUM ([Kpi Goal Reach] * [Conversions]) / 
SUM ([Conversions] * [Has Goal])
END`

We only take into account cases where KPI goal was configured in order to calculate the weighted average for KPI reach. We also put the maximum of 100 KPI reach, which means that the source has achieved the goal. 

- Fraud Score

`SUM([Fraud Index]*[Conversions])/SUM([Conversions])`

Weighted average fraud score, ranges from 0 to 100 (100 being very fraudulent).

Finally, the **Quality Score** is calculated:

`IF isnull([KPI Reach])
THEN ((100-[Fraud Score])+(100-[Clickspam Index]))*0.5
ELSEIF 
isnull([Fraud Score])
THEN ([KPI Reach]+(100-[Clickspam Index]))*0.5
ELSE ((100-[Fraud Score])+[KPI Reach]+(100-[Clickspam Index]))*0.3333
END`

Depending on which metrics are available in each case, we will assign equal weights to each metric, so that the final score ranges between 0 and 100. The higher the quality score, the better this source is for a specific app.

Below you can see the dashboard with the Source Quality Rank. By specifying the app vertical and the OS of the app you would like to promote, as well as the country where you want to promote it, you will get the list of all the sources that can be used for promotion and their corresponding Quality Score.

You can also find the dashboard in my [Tableau Public profile](https://public.tableau.com/views/PQR_0/GlobalSourceQualityRank?:embed=y&:display_count=yes&publish=yes).

In [2]:
%%HTML
<div class='tableauPlaceholder' id='viz1548519156716' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;PQ&#47;PQR_0&#47;GlobalSourceQualityRank&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='PQR_0&#47;GlobalSourceQualityRank' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;PQ&#47;PQR_0&#47;GlobalSourceQualityRank&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1548519156716');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>